In [ ]:
import os
import glob
import sqlite3
import pandas as pd

pasta = '../data'
tipo_arquivo = '*.json'

#Leitura dos aquivos em uma lista
caminhos_arquivos = glob.glob(os.path.join(pasta, tipo_arquivo))

In [ ]:
#Leitura e carga dos arquivos json no banco de dados SQLite
df = pd.DataFrame(columns=['customer', 'flow', 'session', 'timestamp', 'key', 'value'])

conn = sqlite3.connect('meta.db')

for arquivo in caminhos_arquivos:
    data = pd.read_json(arquivo)
    data['key'] = data.apply(lambda x: list(x['content'].keys())[0], axis=1)
    data['value'] = data.apply(lambda x: x['content'][x['key']], axis=1)
    data.drop(columns=['content'], inplace=True, errors='ignore')
    df = pd.concat([df, data], axis=0, ignore_index=False)

# Remove registros nulos e vazios
df['value'] = df['value'].apply(lambda x: None if x.strip() == '' else x)
df.dropna(subset=['value'], inplace=True)
df.head()

df.to_sql('chatbot', conn, if_exists='replace', index=False)
conn.close()

In [ ]:
#Teste do banco de dados
conn = sqlite3.connect('meta.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM chatbot")

data = cursor.fetchall()

select = pd.DataFrame(data, columns=['customer', 'flow', 'session', 'timestamp', 'key', 'value'])

conn.close()

select

In [ ]:
# consulta dos dados aplicando as regras:
# Último valor de cada chave por customer, flow, session e key
# Data de inicio e fim das conversas por customer, flow e session

def consulta_chatbot(customer, flow, session):
    conn = sqlite3.connect('meta.db')
    cursor = conn.cursor()
    cursor.execute(f"""WITH principal as (
               SELECT ROW_NUMBER() OVER (PARTITION BY customer, flow, session, key ORDER BY timestamp DESC) AS row_number,
               customer, 
               flow, 
               session, 
               timestamp, 
               key, 
               value
               FROM chatbot
               WHERE 1=1
               ORDER BY timestamp ASC),
                   
            periodo as (
            SELECT customer, 
                flow,  
                session, 
                min(timestamp) as first_answer_dt, 
                max(timestamp) as last_answer_dt
                FROM chatbot 
                where 1=1
                and flow = '{flow}'
                group by customer, flow, session)

            SELECT p.customer, 
                p.flow,  
                p.session, 
                pe.first_answer_dt as first_answer_dt, 
                pe.last_answer_dt last_answer_dt,  
                p.key,  
                p.value  
                FROM principal p
                left join periodo pe on p.customer = pe.customer and p.flow = pe.flow and p.session = pe.session
                where 1=1
                and p.row_number = 1
                and p.customer = '{customer}'
                and p.flow = '{flow}'
                and p.session = '{session}'
                group by p.customer, p.flow, p.session, p.key
                order by p.timestamp asc""")

    data = cursor.fetchall()
    select = pd.DataFrame(data, columns=['customer', 'flow', 'session', 'first_answer_dt', 'last_answer_dt', 'key', 'value'])
    conn.close()
    return select

In [ ]:
# Teste da função para um unicio caso

data = consulta_chatbot(customer='C1000', flow='F1000', session='S1000')
result_1 = data[['customer', 'flow', 'session', 'first_answer_dt', 'last_answer_dt']].iloc[:1, :]
data = data[['key', 'value']].set_index('key', drop=True)
data = data.T.to_dict('records')
result_2 = pd.DataFrame(data)
result = pd.concat([result_1, result_2], axis=1)
result

In [ ]:
# Teste da função para os casos de flow iguais
# É necessário escolher o flow como apresentado no roteiro do desafio

customers = df['customer'].unique()
flows = df['flow'].unique()
sessions = df['session'].unique()

#######################################
# Seleciona um fluxo específico [0 ou 1]
#######################################

flow = flows[0]
#flow = flows[1]
relatorio = None

for customer in customers:
        for session in sessions:
            data = consulta_chatbot(customer=customer, flow=flow, session=session)
            if data.shape[0] > 0:
                result_1 = data[['customer', 'flow', 'session', 'first_answer_dt', 'last_answer_dt']].iloc[:1, :]
                data = data[['key', 'value']].set_index('key', drop=True)
                data = data.T.to_dict('records')
                result_2 = pd.DataFrame(data)
                result = pd.concat([result_1, result_2], axis=1)
                relatorio = pd.concat([relatorio, result], axis=0, ignore_index=True)

In [ ]:
relatorio